In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import uniform

## Data

In [2]:
%run ../../../../functions/vectorize_functions.py

In [3]:
filepath_name = ("..\\..\\..\\..\\..\\data\\mixed_dataset\\train_cleaned.csv")
df_cleaned = pd.read_csv(filepath_name, encoding='utf-8')

In [4]:
X_train, X_test, y_train, y_test, glove_vectorizer = vectorize_glove(df=df_cleaned, text_column='tweet_cleaned', 
                                                                                 label_column="label")

In [5]:
with open("svm_glove_vectorizer_opt.joblib", "wb") as file:
    joblib.dump(glove_vectorizer, file)

In [6]:
positive = np.count_nonzero(y_train == 1)
negative = np.count_nonzero(y_train == 0)
print("Train")
print("- Positive:", positive)
print("- Negative:", negative)
print("- Verhältnis:", negative / positive)

positive = np.count_nonzero(y_test == 1)
negative = np.count_nonzero(y_test == 0)
print("Test")
print("- Positive:", positive)
print("- Negative:", negative)
print("- Verhältnis:", negative / positive)

Train
- Positive: 10328
- Negative: 47004
- Verhältnis: 4.551123160340821
Test
- Positive: 4417
- Negative: 20155
- Verhältnis: 4.563051845143763


In [7]:
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)

print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (57332, 100)
y_train shape (57332,)
X_test shape (24572, 100)
y_test shape (24572,)


## SVM

In [9]:
# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['sigmoid'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("..\\..\\..\\..\\..\\data\\mixed_dataset\\test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test, _, y_test, _, _ = vectorize_glove(df_test, "tweet", "label", vector_size=100)

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_glove_sigmoid_best_model_opt.joblib", "wb") as file:
    joblib.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_glove_sigmoid_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_glove_sigmoid_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_glove_sigmoid_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_glove_sigmoid_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.8466185005329859, 'recall': 0.7957252588222198, 'f1-score': 0.8203833352461839, 'support': 17966}, '1': {'precision': 0.3715753424657534, 'recall': 0.45588235294117646, 'f1-score': 0.4094339622641509, 'support': 4760}, 'accuracy': 0.7245445744961718, 'macro avg': {'precision': 0.6090969214993697, 'recall': 0.6258038058816981, 'f1-score': 0.6149086487551674, 'support': 22726}, 'weighted avg': {'precision': 0.7471198895851717, 'recall': 0.7245445744961718, 'f1-score': 0.7343092784216447, 'support': 22726}}


In [8]:
# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['linear'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("..\\..\\..\\..\\..\\data\\mixed_dataset\\test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
#X_test, _, y_test, _, _ = vectorize_glove_test_data(df_test, "tweet", "label", vector_size=100)
X_test, _, y_test, _, _ = vectorize_glove(df_test, "tweet", "label", vector_size=100)


# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_glove_linear_best_model_opt.joblib", "wb") as file:
    joblib.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_glove_linear_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_glove_linear_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_glove_linear_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_glove_linear_classification_report_opt.csv", index=True)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.8553334161182168, 'recall': 0.7667816987643327, 'f1-score': 0.8086405259450576, 'support': 17966}, '1': {'precision': 0.36706948640483383, 'recall': 0.5105042016806722, 'f1-score': 0.4270650263620387, 'support': 4760}, 'accuracy': 0.7131039338202939, 'macro avg': {'precision': 0.6112014512615254, 'recall': 0.6386429502225024, 'f1-score': 0.6178527761535482, 'support': 22726}, 'weighted avg': {'precision': 0.7530656916864777, 'recall': 0.7131039338202939, 'f1-score': 0.7287188777000884, 'support': 22726}}


In [9]:
# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['poly'],
    'degree': [3, 4, 5],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("..\\..\\..\\..\\..\\data\\mixed_dataset\\test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
#X_test, _, y_test, _, _ = vectorize_glove_test_data(df_test, "tweet", "label", vector_size=100)
X_test, _, y_test, _, _ = vectorize_glove(df_test, "tweet", "label", vector_size=100)


# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_glove_poly_best_model_opt.joblib", "wb") as file:
    joblib.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_glove_poly_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_glove_poly_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_glove_poly_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_glove_poly_classification_report_opt.csv", index=True)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.8430747623163354, 'recall': 0.8686964265835467, 'f1-score': 0.8556938428641921, 'support': 17966}, '1': {'precision': 0.44019933554817275, 'recall': 0.3897058823529412, 'f1-score': 0.41341653666146644, 'support': 4760}, 'accuracy': 0.7683710287776115, 'macro avg': {'precision': 0.641637048932254, 'recall': 0.629201154468244, 'f1-score': 0.6345551897628293, 'support': 22726}, 'weighted avg': {'precision': 0.758691807488541, 'recall': 0.7683710287776115, 'f1-score': 0.763058096251283, 'support': 22726}}


In [10]:
# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['rbf'],
    'degree': [3, 4, 5],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("..\\..\\..\\..\\..\\data\\mixed_dataset\\test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
#X_test, _, y_test, _, _ = vectorize_glove_test_data(df_test, "tweet", "label", vector_size=100)
X_test, _, y_test, _, _ = vectorize_glove(df_test, "tweet", "label", vector_size=100)


# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_glove_rbf_best_model_opt.joblib", "wb") as file:
    joblib.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_glove_rbf_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_glove_rbf_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_glove_rbf_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_glove_rbf_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.8293510250910361, 'recall': 0.9254146721585217, 'f1-score': 0.8747533738457897, 'support': 17966}, '1': {'precision': 0.49981336319522207, 'recall': 0.28130252100840336, 'f1-score': 0.35999462293318996, 'support': 4760}, 'accuracy': 0.7905042682390214, 'macro avg': {'precision': 0.6645821941431291, 'recall': 0.6033585965834625, 'f1-score': 0.6173739983894898, 'support': 22726}, 'weighted avg': {'precision': 0.7603287919385203, 'recall': 0.7905042682390214, 'f1-score': 0.7669362632964641, 'support': 22726}}
